Устанавливаем библиотеки: pandas - для чтения excel и работы с таблицами, plotly - для рисования красивых интерактивных графиков

Обязательно указываем версию библиотеки через ==, потому что код, работающий в одной версии, может не работать в другой.
Документации на них:

https://pandas.pydata.org

https://plotly.com/python/


In [1]:
!pip install pandas==1.4.4 plotly==5.13.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from itertools import tee # https://docs.python.org/3/library/itertools.html#itertools.tee

import pandas as pd

In [3]:
# подключаем Google диск
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Построение графиков изменения среднесписочной численности работников организаций (без субъектов малого предпринимательства) (с 2017 года)

In [4]:
# здесь нужно будет изменить название города на ваш, как в папке с данными
mean_numer_of_employees = pd.read_excel('drive/MyDrive/БДМО/Якутск/number_of_employees.xlsx')
mean_numer_of_employees

,Показатели,Ед. измерения,2017,2018,2019,2020,2021,2022
0,Среднесписочная численность работников организ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Всего по обследуемым видам экономической деяте...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,январь-март,человек,83545.0,82168.0,83635.0,84602.0,84673.0,85579.0
3,январь-июнь,человек,83617.0,82687.0,84411.0,84654.0,84903.0,85730.0
4,январь-сентябрь,человек,83465.0,83046.0,84604.0,84724.0,85144.0,85714.0
...,...,...,...,...,...,...,...,...
96,Раздел S Предоставление прочих видов услуг,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,январь-март,человек,345.0,355.0,248.0,255.0,210.0,146.0
98,январь-июнь,человек,408.0,343.0,250.0,252.0,207.0,153.0
99,январь-сентябрь,человек,433.0,349.0,257.0,249.0,227.0,158.0


In [5]:
!python --version # какая на Colab версия Python

Python 3.9.16


In [6]:
def pairwise(iterable): # начиная с версии Python 3.10 реализована в библиотеке itertools . у меня на Colab команда выше сейчас показывает Python 3.9
    # pairwise('ABCDEFG') --> AB BC CD DE EF FG
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

In [7]:
# все графики будем строить по данным за календарный год (январь-декабрь). По данным до марта, июня и сентября можно было бы попробовать изучить сезонные изменения (например, проверить 
# предположение, что летом становится больше работников в сельскохозяйственной отрасли). Но в данном домашнем задании такая задача не стоит.
# Составим новую таблицу, где будут только данные за год.

# соберем все строки из столбца "Показатели", которые содержат в себе слово "Раздел"

division = mean_numer_of_employees["Показатели"][mean_numer_of_employees["Показатели"].str.contains("Раздел")]
whole_year = pd.DataFrame()

# нужно пройтись по всем видам экономической деятельности и собрать данные за январь-декабрь. В некоторых видах есть, например, данные только за январь-март
# такие виды экономической деятельности не используем

# граничный случай 1 - первый в таблице вид экономической деятельности
# в row_names будут имена строк

row_names = ['Всего по обследуемым видам экономической деятельности']
current_economic_activity = mean_numer_of_employees.iloc[:division.index[0]]
print(current_economic_activity)
# создаем маску, чтобы получить таблицу с данными только по календарным годам.
mask = current_economic_activity['Показатели'].str.lower() == 'январь-декабрь' # в Тульской области месяцы с заглавной буквы, в остальных со строчной
# поэтому преобразуем все буквы в строчные
# dtype: bool - тип данных логический, от Булева логика("истинно" или "ложно"). 
print(mask.head())
# применяем маску - используем только те строчки, где значения маски True
calendar_year = current_economic_activity[mask]
# Для желающих разбираться: туториал простыми словами по выделению отдельных строчек из pandas DataFrame 
# https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html#how-do-i-filter-specific-rows-from-a-dataframe
# И более формальная документация: https://pandas.pydata.org/docs/user_guide/indexing.html?highlight=selection#selection-by-label
if not calendar_year.empty: 
    # добавляем строчку к новой таблице
    whole_year = pd.concat((whole_year, calendar_year))

for pair in pairwise(division.index):
    print(pair)
    
    # отбираем строчки, относящиеся к данному виду экономической деятельности
    current_economic_activity = mean_numer_of_employees.iloc[pair[0]:pair[1]]
    mask = current_economic_activity['Показатели'].str.lower() == 'январь-декабрь'
    calendar_year = current_economic_activity[mask]
    if not calendar_year.empty: 
        row_names.append(mean_numer_of_employees["Показатели"][pair[0]])
        whole_year = pd.concat((whole_year, calendar_year))

# граничный случай 2 - последний вид экономической деятельности
current_economic_activity = mean_numer_of_employees.iloc[division.index[-1]:]
mask = current_economic_activity['Показатели'].str.lower() == 'январь-декабрь' # в Тульской области месяцы с заглавной буквы, в остальных со строчной
calendar_year = current_economic_activity[mask]
if not calendar_year.empty: 
    row_names.append(mean_numer_of_employees["Показатели"][division.index[-1]])
    whole_year = pd.concat((whole_year, calendar_year))
print(f"Длина списка, который вставляем как столбец: {len(row_names)}") 
whole_year.insert(0, 'Вид экономической деятельности', row_names)
whole_year

                                          Показатели Ед. измерения     2017  \
0  Среднесписочная численность работников организ...           NaN      NaN   
1  Всего по обследуемым видам экономической деяте...           NaN      NaN   
2                                        январь-март       человек  83545.0   
3                                        январь-июнь       человек  83617.0   
4                                    январь-сентябрь       человек  83465.0   
5                                     январь-декабрь       человек  83139.7   

      2018     2019     2020     2021     2022  
0      NaN      NaN      NaN      NaN      NaN  
1      NaN      NaN      NaN      NaN      NaN  
2  82168.0  83635.0  84602.0  84673.0  85579.0  
3  82687.0  84411.0  84654.0  84903.0  85730.0  
4  83046.0  84604.0  84724.0  85144.0  85714.0  
5  83144.0  84503.0  84258.0  85275.0  85396.0  
0    False
1    False
2    False
3    False
4    False
Name: Показатели, dtype: bool
(6, 11)
(11, 16)
(

,Вид экономической деятельности,Показатели,Ед. измерения,2017,2018,2019,2020,2021,2022
5,Всего по обследуемым видам экономической деяте...,январь-декабрь,человек,83139.7,83144.0,84503.0,84258.0,85275.0,85396.0
10,"Раздел А Сельское, лесное хозяйство, охота, ры...",январь-декабрь,человек,940.9,864.0,932.0,850.0,901.0,1006.0
15,Раздел В Добыча полезных ископаемых,январь-декабрь,человек,812.2,1042.0,1169.0,1155.0,1071.0,1164.0
20,Раздел C Обрабатывающие производства,январь-декабрь,человек,2025.6,1879.0,2055.0,1774.0,1642.0,1589.0
25,"Раздел D Обеспечение электрической энергией, г...",январь-декабрь,человек,5795.0,4878.0,5028.0,4675.0,4791.0,4551.0
30,"Раздел Е Водоснабжение; водоотведение, организ...",январь-декабрь,человек,NaN,993.0,1131.0,1162.0,1284.0,1317.0
35,Раздел F Строительство,январь-декабрь,человек,1711.0,1016.0,964.0,811.0,874.0,1153.0
40,Раздел G торговля оптовая и розничная; ремонт ...,январь-декабрь,человек,3137.2,3205.0,3016.0,3246.0,3529.0,3396.0
45,Раздел Н Транспортировка и хранение,январь-декабрь,человек,7724.8,8507.0,8716.0,8382.0,8301.0,8198.0
50,Раздел I Деятельность гостиниц и предприятий о...,январь-декабрь,человек,NaN,673.0,645.0,551.0,459.0,457.0


In [8]:
import plotly.express as px

In [9]:
# чтобы передать данные в plotly, понадобится изменить формат таблицы

numbers_of_employees, years = [], []
# не берем строчку всего, т.к. там значения сильно больше, масштаб графика будет неудобный для просмотра
economic_activities = whole_year[1:]['Вид экономической деятельности'].to_list() * 6 

for year in range(2017, 2023):
    numbers_of_employees = numbers_of_employees + whole_year[1:][year].to_list()
    years = years + [year for _ in range(whole_year[1:].shape[0])] # как _ принято называть переменную, которая не используется далее

plotly_data = pd.DataFrame(
    {
    "Годы": years,
    "Среднесписочная численность работников": numbers_of_employees,
    'Вид экономической деятельности': economic_activities,
    },
)

In [10]:
fig = px.line(
    plotly_data,
    x='Годы',
    y='Среднесписочная численность работников',
    color='Вид экономической деятельности',
    markers=True,
    )
fig.show()
# пропущенные в таблице значения на графике просто не отображаются, например, для Якутска пропущено значение для 2017 года по общепиту и этот график начинается с 2018

На графике выше слабо видны различия по годам для видов экономической деятельности с численностью работников менее 2 тысяч. Подсчитаем и отрисуем показатели динамики - базисный и цепной темпы роста.

Базисный показатель динамики - такой, в котором сравниваем значение текущего уровня временного ряда с одним и тем же уровнем, принятым за базу. В нашем случае базисным уровнем выбираем 2018 год (так как в данных для Якутска для 2017 года некоторые значения пропущены).



In [11]:
# Базисный темп роста
numbers_of_employees, years = [], []
# не берем строчку всего, т.к. там значения сильно больше, масштаб графика будет неудобный для просмотра
economic_activities = whole_year[1:]['Вид экономической деятельности'].to_list() * 5

for year in range(2018, 2023):
    numbers_of_employees = numbers_of_employees + ((whole_year[1:][year]/whole_year[1:][2018]) * 100).to_list()
    years = years + [year for _ in range(whole_year[1:].shape[0])] # как _ принято называть переменную, которая не используется далее

plotly_data = pd.DataFrame(
    {
    "Годы": years,
    "Базисный темп роста среднесписочной численности работников, %": numbers_of_employees,
    'Вид экономической деятельности': economic_activities,
    },
)
fig = px.line(
    plotly_data,
    x='Годы',
    y='Базисный темп роста среднесписочной численности работников, %',
    color='Вид экономической деятельности',
    markers=True,
    )
fig.show()
# примечание. При описании графика удобно наводить курсор на точки соответствующих линий, так как некоторые цвета слабо различимы.

Видно, что после 2020 года темп роста был наименьшим (численность работников составила 50% от базисного значения 2018 года) у гостиниц и предприятий общественного питания. Интересно, что случилось в 2020 году?

Цепной показатель динамики - такой, в котором сравниваем значение текущего уровня временного ряда с предыдущим. То есть 2019 год с 2018-м, 2020 с 2019-м, и так далее.

In [12]:
# Цепной темп роста
numbers_of_employees, years = [], []
# не берем строчку всего, т.к. там значения сильно больше, масштаб графика будет неудобный для просмотра
economic_activities = whole_year[1:]['Вид экономической деятельности'].to_list() * 4

for year in range(2019, 2023):
    numbers_of_employees = numbers_of_employees + ((whole_year[1:][year]/whole_year[1:][year-1]) * 100).to_list()
    years = years + [year for _ in range(whole_year[1:].shape[0])] # как _ принято называть переменную, которая не используется далее

plotly_data = pd.DataFrame(
    {
    "Годы": years,
    "Цепной темп роста среднесписочной численности работников, %": numbers_of_employees,
    'Вид экономической деятельности': economic_activities,
    },
)
fig = px.line(
    plotly_data,
    x='Годы',
    y='Цепной темп роста среднесписочной численности работников, %',
    color='Вид экономической деятельности',
    markers=True,
    )
fig.show()
# примечание. При описании графика удобно наводить курсор на точки соответствующих линий, так как некоторые цвета слабо различимы.
# чтобы скачать картинку, надо навести курсор на значок фотоаппарата в верхнем правом углу и щелкнуть

На этом графике видно, что в 2022 году численность работников гостиниц и общепита сократилась по сравнению с 2021 годом незначительно (на 0.4 %).
Но помним, что численность их всё ещё продолжает уменьшаться и удерживаться на невысоком уровне в сравнении с 2018 годом (по графику базисного темпа прироста).

# Построение графиков изменения среднемесячной заработной платы работников организаций (без субъектов малого предпринимательства) с 2017 г.

In [13]:
# здесь нужно будет изменить путь до вашего файла с данными
mean_salary = pd.read_excel('drive/MyDrive/БДМО/Якутск/mean_salary.xlsx')
division = mean_numer_of_employees["Показатели"][mean_numer_of_employees["Показатели"].str.contains("Раздел")]
whole_year = pd.DataFrame()

row_names = ['Всего по обследуемым видам экономической деятельности']
current_economic_activity = mean_numer_of_employees.iloc[:division.index[0]]
mask = current_economic_activity['Показатели'].str.lower() == 'январь-декабрь' 
calendar_year = current_economic_activity[mask]
if not calendar_year.empty: 
    whole_year = pd.concat((whole_year, calendar_year))

for pair in pairwise(division.index):
    current_economic_activity = mean_numer_of_employees.iloc[pair[0]:pair[1]]
    mask = current_economic_activity['Показатели'].str.lower() == 'январь-декабрь'
    calendar_year = current_economic_activity[mask]
    if not calendar_year.empty: 
        row_names.append(mean_numer_of_employees["Показатели"][pair[0]])
        whole_year = pd.concat((whole_year, calendar_year))

current_economic_activity = mean_numer_of_employees.iloc[division.index[-1]:]
mask = current_economic_activity['Показатели'].str.lower() == 'январь-декабрь'
calendar_year = current_economic_activity[mask]
if not calendar_year.empty: 
    row_names.append(mean_numer_of_employees["Показатели"][division.index[-1]])
    whole_year = pd.concat((whole_year, calendar_year))
print(f"Длина списка, который вставляем как столбец: {len(row_names)}") 
whole_year.insert(0, 'Вид экономической деятельности', row_names)
whole_year

Длина списка, который вставляем как столбец: 20


,Вид экономической деятельности,Показатели,Ед. измерения,2017,2018,2019,2020,2021,2022
5,Всего по обследуемым видам экономической деяте...,январь-декабрь,человек,83139.7,83144.0,84503.0,84258.0,85275.0,85396.0
10,"Раздел А Сельское, лесное хозяйство, охота, ры...",январь-декабрь,человек,940.9,864.0,932.0,850.0,901.0,1006.0
15,Раздел В Добыча полезных ископаемых,январь-декабрь,человек,812.2,1042.0,1169.0,1155.0,1071.0,1164.0
20,Раздел C Обрабатывающие производства,январь-декабрь,человек,2025.6,1879.0,2055.0,1774.0,1642.0,1589.0
25,"Раздел D Обеспечение электрической энергией, г...",январь-декабрь,человек,5795.0,4878.0,5028.0,4675.0,4791.0,4551.0
30,"Раздел Е Водоснабжение; водоотведение, организ...",январь-декабрь,человек,NaN,993.0,1131.0,1162.0,1284.0,1317.0
35,Раздел F Строительство,январь-декабрь,человек,1711.0,1016.0,964.0,811.0,874.0,1153.0
40,Раздел G торговля оптовая и розничная; ремонт ...,январь-декабрь,человек,3137.2,3205.0,3016.0,3246.0,3529.0,3396.0
45,Раздел Н Транспортировка и хранение,январь-декабрь,человек,7724.8,8507.0,8716.0,8382.0,8301.0,8198.0
50,Раздел I Деятельность гостиниц и предприятий о...,январь-декабрь,человек,NaN,673.0,645.0,551.0,459.0,457.0


In [14]:
salaries, years = [], []
# не берем строчку всего, т.к. там значения сильно больше, масштаб графика будет неудобный для просмотра
economic_activities = whole_year[1:]['Вид экономической деятельности'].to_list() * 6 

for year in range(2017, 2023):
    salaries = salaries + whole_year[1:][year].to_list()
    years = years + [year for _ in range(whole_year[1:].shape[0])] # как _ принято называть переменную, которая не используется далее

plotly_data = pd.DataFrame(
    {
    "Годы": years,
    "Среднемесячная зарплата": salaries,
    'Вид экономической деятельности': economic_activities,
    },
)
fig = px.line(
    plotly_data,
    x='Годы',
    y='Среднемесячная зарплата',
    color='Вид экономической деятельности',
    markers=True,
    )
fig.show()

In [15]:
# Базисный темп роста
salaries, years = [], []
# не берем строчку всего, т.к. там значения сильно больше, масштаб графика будет неудобный для просмотра
economic_activities = whole_year[1:]['Вид экономической деятельности'].to_list() * 5

for year in range(2018, 2023):
    salaries = salaries + ((whole_year[1:][year]/whole_year[1:][2018]) * 100).to_list()
    years = years + [year for _ in range(whole_year[1:].shape[0])] # как _ принято называть переменную, которая не используется далее

plotly_data = pd.DataFrame(
    {
    "Годы": years,
    "Базисный темп роста среднемесячной зарплаты, %": salaries,
    'Вид экономической деятельности': economic_activities,
    },
)
fig = px.line(
    plotly_data,
    x='Годы',
    y='Базисный темп роста среднемесячной зарплаты, %',
    color='Вид экономической деятельности',
    markers=True,
    )
fig.show()
# примечание. При описании графика удобно наводить курсор на точки соответствующих линий, так как некоторые цвета слабо различимы.

In [16]:
# Цепной темп роста
salaries, years = [], []
# не берем строчку всего, т.к. там значения сильно больше, масштаб графика будет неудобный для просмотра
economic_activities = whole_year[1:]['Вид экономической деятельности'].to_list() * 4

for year in range(2019, 2023):
    salaries = salaries + ((whole_year[1:][year]/whole_year[1:][year-1]) * 100).to_list()
    years = years + [year for _ in range(whole_year[1:].shape[0])] # как _ принято называть переменную, которая не используется далее

plotly_data = pd.DataFrame(
    {
    "Годы": years,
    "Цепной темп роста среднемесячной зарплаты, %": salaries,
    'Вид экономической деятельности': economic_activities,
    },
)
fig = px.line(
    plotly_data,
    x='Годы',
    y='Цепной темп роста среднемесячной зарплаты, %',
    color='Вид экономической деятельности',
    markers=True,
    )
fig.show()
# примечание. При описании графика удобно наводить курсор на точки соответствующих линий, так как некоторые цвета слабо различимы.
# чтобы скачать картинку, надо навести курсор на значок фотоаппарата в верхнем правом углу и щелкнуть